## About the competition
The goal of this competition is to predict the outcome of March Madness matches.This compeptition comes in two stages,
* stage 1, we have to predict the winner and looser for historical data for the given team ID pairs.
* stage 2, we have to find outcomes for 2021(current year matches) tournament.
</br>

We are going to build a simple logistic model that is heavily adapted from great kernel by [Radder from here](https://www.kaggle.com/raddar/team-power-rankings). First we will do some feature engineering based on team data.
The aim of this notebook is to build a simple baseline model for prediction of historical data(stage 1).
In later versions, we'll try new things and iterate on this version to make more complex models/or use more features.

## Acknowledgement:
checkout these great kernels which showscase interesting idea related to the comp.:           
[ncaam-2021-eda-all-you-need-to-know](https://www.kaggle.com/dhananjay3/ncaam-2021-eda-all-you-need-to-know)           
[ncaa-starter-the-simpler-the-better](https://www.kaggle.com/theoviel/ncaa-starter-the-simpler-the-better)             
[2021-ncaam-tournament-rapids-starter](https://www.kaggle.com/tunguz/2021-ncaam-tournament-rapids-starter)

## Importing libraries

In [1]:
import os
import re
import sklearn
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter
import statsmodels.api as sm
from sklearn.metrics import *
from sklearn.linear_model import *
from sklearn.model_selection import *
from sklearn.utils import shuffle
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier


### Looking into data directory

In [2]:
DATA_PATH = '../input/ncaam-march-mania-2021/'

for filename in os.listdir(DATA_PATH):
    print(filename)

Conferences.csv
MNCAATourneySeedRoundSlots.csv
MRegularSeasonDetailedResults.csv
MNCAATourneyCompactResults.csv
MGameCities.csv
MSeasons.csv
MSecondaryTourneyTeams.csv
Cities.csv
MTeamSpellings.csv
MRegularSeasonCompactResults.csv
MMasseyOrdinals.csv
MSecondaryTourneyCompactResults.csv
MNCAATourneySlots.csv
MNCAATourneySeeds.csv
MTeamCoaches.csv
MConferenceTourneyGames.csv
MNCAATourneyDetailedResults.csv
MSampleSubmissionStage1.csv
MTeamConferences.csv
MTeams.csv


### Importing team seeds data
Seeds are unique identifiers for each team, assigned based on the performace estimation,
the first character is either W, X, Y, or Z identifying the region the team was in and the next two digits 01, 02, to 15, 16 tell you the seed within the region

In [3]:
seeds_path = '../input/ncaam-march-mania-2021/MNCAATourneySeeds.csv'
df_seed = pd.read_csv(seeds_path)
df_seed

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374
...,...,...,...
2281,2019,Z12,1332
2282,2019,Z13,1414
2283,2019,Z14,1330
2284,2019,Z15,1159


### Regular Season Results

This file contains data for regular season and gives significant understanding of outcomes( details results give more information but we are starting with compact)

In [4]:
df_comp_results = pd.read_csv('../input/ncaam-march-mania-2021/MRegularSeasonCompactResults.csv')
df_comp_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


Check for Null values in training data and overview of each year's data

In [5]:
df_comp_results.groupby("Season").describe()

DayNum                                                       WTeamID  \
         count       mean        std   min   25%   50%    75%    max   count   
Season                                                                         
1985    3737.0  81.817233  29.640793  20.0  60.0  84.0  107.0  132.0  3737.0   
1986    3783.0  81.445678  29.729152  24.0  56.0  82.0  107.0  132.0  3783.0   
1987    3915.0  82.907535  28.687774  25.0  61.0  84.0  107.0  132.0  3915.0   
1988    3955.0  79.438938  30.339697  18.0  56.0  82.0  105.0  132.0  3955.0   
1989    4037.0  79.123854  30.523473  18.0  53.0  82.0  105.0  132.0  4037.0   
1990    4045.0  79.195303  30.509001  16.0  54.0  82.0  105.0  132.0  4045.0   
1991    4123.0  78.725928  30.613389  16.0  53.5  82.0  104.0  132.0  4123.0   
1992    4127.0  76.514660  31.695209  11.0  53.0  79.0  103.0  132.0  4127.0   
1993    3982.0  81.057258  28.826007  16.0  58.0  82.0  105.0  132.0  3982.0   
1994    4060.0  80.279557  29.892168  16.0  58.0  82.0  105.0  132.0  4060.0   
1995    4077.0  79.828550  30.085734  16.0  58.0  82.0  105.0  132.0  4077.0   
1996    4122.0  79.872392  30.279643  16.0  58.0  82.0  105.0  132.0  4122.0   
1997    4155.0  80.003369  29.957344  18.0  55.0  82.0  105.0  132.0  4155.0   
1998    4167.0  78.013679  31.669471  15.0  52.0  81.0  104.0  132.0  4167.0   
1999    4222.0  76.980341  32.164855  12.0  47.0  80.0  103.0  132.0  4222.0   
2000    4519.0  75.169285  32.736731  10.0  47.0  77.0  103.0  132.0  4519.0   
2001    4467.0  75.721289  32.771434  10.0  47.0  78.0  103.0  132.0  4467.0   
2002    4555.0  75.277497  33.132144  10.0  47.0  78.0  103.0  132.0  4555.0   
2003    4616.0  74.808709  32.748115  10.0  47.0  75.0  103.0  132.0  4616.0   
2004    4571.0  75.335156  32.683224  10.0  47.0  77.0  103.0  132.0  4571.0   
2005    4675.0  75.042567  33.071668  10.0  47.0  77.0  103.0  132.0  4675.0   
2006    4757.0  74.712634  33.155336   8.0  47.0  77.0  103.0  132.0  4757.0   
2007    5043.0  71.676780  34.986831   8.0  40.0  75.0  102.0  132.0  5043.0   
2008    5163.0  68.392020  37.173746   0.0  33.0  72.0  100.0  132.0  5163.0   
2009    5249.0  70.963612  35.107186   7.0  40.0  73.0  101.0  132.0  5249.0   
2010    5263.0  70.741212  35.184758   7.0  40.0  73.0  101.0  132.0  5263.0   
2011    5246.0  70.881243  35.348692   7.0  40.0  73.0  101.0  132.0  5246.0   
2012    5253.0  70.745479  35.224383   7.0  40.0  73.0  101.0  132.0  5253.0   
2013    5320.0  68.058647  37.270185   4.0  33.0  72.0  100.0  132.0  5320.0   
2014    5362.0  68.882507  37.423343   4.0  33.0  73.0  101.0  132.0  5362.0   
2015    5354.0  70.742249  35.421389  11.0  40.0  73.0  101.0  132.0  5354.0   
2016    5369.0  70.922332  35.477876  11.0  40.0  73.0  102.0  132.0  5369.0   
2017    5395.0  70.796849  35.602035  11.0  37.0  73.0  102.0  132.0  5395.0   
2018    5405.0  70.756337  35.526140  11.0  37.0  73.0  101.0  132.0  5405.0   
2019    5463.0  67.501007  38.309412   1.0  33.0  71.0  101.0  132.0  5463.0   
2020    5328.0  66.265390  37.303726   1.0  32.0  69.0   98.0  128.0  5328.0   

                     ... LScore          NumOT                                \
               mean  ...    75%    max   count      mean       std  min  25%   
Season               ...                                                       
1985    1287.701097  ...   70.0  140.0  3737.0  0.007760  0.104470  0.0  0.0   
1986    1285.545863  ...   70.0  114.0  3783.0  0.002115  0.051377  0.0  0.0   
1987    1284.852107  ...   74.0  111.0  3915.0  0.008429  0.116061  0.0  0.0   
1988    1284.863970  ...   75.0  133.0  3955.0  0.003793  0.065460  0.0  0.0   
1989    1283.772851  ...   77.0  150.0  4037.0  0.005697  0.081592  0.0  0.0   
1990    1282.814339  ...   77.0  141.0  4045.0  0.006180  0.092826  0.0  0.0   
1991    1287.633519  ...   78.0  140.0  4123.0  0.007519  0.099449  0.0  0.0   
1992    1285.796947  ...   75.0  116.0  4127.0  0.003150  0.071272  0.0  0.0   
1993   

Above statistics shows zero null values with some important conclusions:
* we can notice that there is approx. 10 points difference between mean Wscore and Lscore, with both having std of around 10.7
* The data contains 1985 to 2019 all seasons data with some variation in number of teams
* there is no particular relationship with Winning team ID and losing team ID
* The dayNum defines what day the game was played on, ranging from 0 to 132 
* WLoc gives idea about location of match w.r.t. winning team (H for Home, A for Away, N for Neutral)
* WTeamID and LTeamID describes winning and loosing team ID respectively

### Importing Tournament Results

This data comprises of tournament results from 1985 to 2019 in the same format as that of Regular season results

In [6]:
df_trny_results = pd.read_csv('../input/ncaam-march-mania-2021/MSecondaryTourneyCompactResults.csv')
df_trny_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,SecondaryTourney
0,1985,136,1151,67,1155,65,H,0,NIT
1,1985,136,1153,77,1245,61,H,0,NIT
2,1985,136,1201,79,1365,76,H,0,NIT
3,1985,136,1231,79,1139,57,H,0,NIT
4,1985,136,1249,78,1222,71,H,0,NIT


### Importing sample submission for stage 1( Historical data)

In [7]:
test_data = pd.read_csv('../input/ncaam-march-mania-2021/MSampleSubmissionStage1.csv')
test_data.head()

,ID,Pred
0,2015_1107_1112,0.5
1,2015_1107_1116,0.5
2,2015_1107_1124,0.5
3,2015_1107_1125,0.5
4,2015_1107_1129,0.5


We can see that data for historical seasons, start from year 2015 so that in order to avoid leaking we are neglecting data after 2015 for training.



In [8]:
#submission preprocessing 
test_data['Season'] = test_data['ID'].apply(lambda x: int(x.split('_')[0]))
test_data['TeamIdA'] = test_data['ID'].apply(lambda x: int(x.split('_')[1]))
test_data['TeamIdB'] = test_data['ID'].apply(lambda x: int(x.split('_')[2]))

## Feature Engineering

we will try to rename winning and loosing team with T1 and T2 for better understanding.

In [9]:
#this feature engineering is adapted from this great kernal
# team power ranking by radddar, https://www.kaggle.com/raddar/team-power-rankings
def prepare_data(df):
    dfswap = df[['Season', 'DayNum', 'LTeamID', 'LScore', 'WTeamID', 'WScore', 'WLoc', 'NumOT']]

    dfswap.loc[df['WLoc'] == 'H', 'WLoc'] = 'A'
    dfswap.loc[df['WLoc'] == 'A', 'WLoc'] = 'H'
    df.columns.values[6] = 'location'
    dfswap.columns.values[6] = 'location'         
    df.columns = [x.replace('W','T1_').replace('L','T2_') for x in list(df.columns)]
    dfswap.columns = [x.replace('L','T1_').replace('W','T2_') for x in list(dfswap.columns)]
    output = pd.concat([df, dfswap]).sort_index().reset_index(drop=True)
    
    return output

In [10]:
df_trny_results = prepare_data(df_trny_results)
df_comp_results = prepare_data(df_comp_results)

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1719: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [11]:
df_comp_results.head()

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,20,1328,64,1228,81,N,0
2,1985,25,1354,70,1106,77,A,0
3,1985,25,1106,77,1354,70,H,0
4,1985,25,1112,63,1223,56,H,0


The next section describes our model with given formula which essentially tries to capture the quality or estimated rank of a team, so it will give us a general idea of a teams performace (which signifies chance of winning in other terms)

In [12]:
df_comp_results['T1_TeamID'] = df_comp_results['T1_TeamID'].astype('str')
df_comp_results['T2_TeamID'] = df_comp_results['T2_TeamID'].astype('str')


# make it a binary task
df_comp_results['win'] = np.where(df_comp_results['T1_Score']>df_comp_results['T2_Score'], 1, 0)
def team_quality(season):
    """
    Calculate team quality for each season seperately. 
    Team strength changes from season to season (students playing change!)
    So pooling everything would be bad approach!
    """
    formula = 'win~-1+T1_TeamID+T2_TeamID'
    glm = sm.GLM.from_formula(formula=formula, 
                              data=df_comp_results.loc[df_comp_results.Season==season,:], 
                              family=sm.families.Binomial()).fit()
    
    # extracting parameters from glm
    quality = pd.DataFrame(glm.params).reset_index()
    quality.columns = ['TeamID','beta']
    quality['Season'] = season
    # taking exp due to binomial model being used
    quality['quality'] = np.exp(quality['beta'])
    # only interested in glm parameters with T1_, as T2_ should be mirroring T1_ ones
    quality = quality.loc[quality.TeamID.str.contains('T1_')].reset_index(drop=True)
    quality['TeamID'] = quality['TeamID'].apply(lambda x: x[10:14]).astype(int)
    return quality

In [13]:
team_quality = pd.concat([team_quality(2010),
                          team_quality(2011),
                          team_quality(2012),
                          team_quality(2013),
                          team_quality(2014),
                          team_quality(2015),
                          team_quality(2016),
                          team_quality(2017),
                          team_quality(2018),
                          team_quality(2019)]).reset_index(drop=True)

In [14]:
#get qualities separately for each team
team_quality_T1 = team_quality[['TeamID','Season','quality']]
team_quality_T1.columns = ['T1_TeamID','Season','T1_quality']
team_quality_T2 = team_quality[['TeamID','Season','quality']]
team_quality_T2.columns = ['T2_TeamID','Season','T2_quality']


df_trny_results['T1_TeamID'] = df_trny_results['T1_TeamID'].astype(int)
df_trny_results['T2_TeamID'] = df_trny_results['T2_TeamID'].astype(int)
df_trny_results = df_trny_results.merge(team_quality_T1, on = ['T1_TeamID','Season'], how = 'left')
df_trny_results = df_trny_results.merge(team_quality_T2, on = ['T2_TeamID','Season'], how = 'left')



In [15]:
df_seed['seed'] = df_seed['Seed'].apply(lambda x: int(x[1:3]))
df_seed['division'] = df_seed['Seed'].apply(lambda x: x[0])

seeds_T1 = df_seed[['Season','TeamID','seed','division']].copy()
seeds_T2 = df_seed[['Season','TeamID','seed','division']].copy()
seeds_T1.columns = ['Season','T1_TeamID','T1_seed','T1_division']
seeds_T2.columns = ['Season','T2_TeamID','T2_seed','T2_division']

df_trny_results = df_trny_results.merge(seeds_T1, on = ['Season', 'T1_TeamID'], how = 'left')
df_trny_results = df_trny_results.merge(seeds_T2, on = ['Season', 'T2_TeamID'], how = 'left')

In [16]:
test_range = df_trny_results['Season'].between(2015,2019)

In [17]:
df_trny_results[test_range].groupby(['Season','T1_division'])['T1_quality'].rank(method='dense', ascending=False)

2506    208.0
2507    184.0
2508     17.0
2509     36.0
2510    230.0
        ...  
3243    330.0
3244    275.0
3245    180.0
3246    301.0
3247    265.0
Name: T1_quality, Length: 742, dtype: float64

In [18]:
df_trny_results['T1_powerrank'] = df_trny_results[test_range].groupby(['Season','T1_division'])['T1_quality'].rank(method='dense', ascending=False).astype(int)
df_trny_results['T2_powerrank'] = df_trny_results[test_range].groupby(['Season','T2_division'])['T2_quality'].rank(method='dense', ascending=False).astype(int)

## Conclusion
This work is still incomplete as we will further filter this powerranks as according to our test data and train a model based on the new feature (in next versions), but hopefull this notebook will give you concept of work which can be useful for this competition. 
